In [89]:
import pandas as pd

In [90]:
weather = pd.read_parquet("bcn_temp_prec.parquet")
weather = weather[['valid_time', 't2m', 'tp']]
weather

,valid_time,t2m,tp
0,2019-01-01 00:00:00,282.796143,0.000000e+00
1,2019-01-01 06:00:00,282.968506,0.000000e+00
2,2019-01-01 12:00:00,287.306641,0.000000e+00
3,2019-01-01 18:00:00,285.851074,0.000000e+00
4,2019-01-02 00:00:00,282.264893,1.904875e-07
...,...,...,...
9415,2025-06-11 18:00:00,297.439697,0.000000e+00
9416,2025-06-12 00:00:00,295.605469,0.000000e+00
9417,2025-06-12 06:00:00,295.270996,0.000000e+00
9418,2025-06-12 12:00:00,297.957764,0.000000e+00


In [101]:
from pathlib import Path

carpeta = Path("data_energia")
archivos = carpeta.glob("*.csv")
df = pd.concat((pd.read_csv(archivo) for archivo in archivos), ignore_index=True)

df

,Any,Data,Codi_Postal,Sector_Economic,Tram_Horari,Valor
0,2019,2019-01-01,8001,Indústria,De 00:00:00 a 05:59:59 h,727
1,2019,2019-01-01,8001,Indústria,De 06:00:00 a 11:59:59 h,855
2,2019,2019-01-01,8001,Indústria,De 12:00:00 a 17:59:59 h,1175
3,2019,2019-01-01,8001,Indústria,De 18:00:00 a 23:59:59 h,1214
4,2019,2019-01-01,8001,Indústria,No consta,106
...,...,...,...,...,...,...
1493135,2025,2025-02-28,8042,Serveis,De 00:00:00 a 05:59:59 h,6132
1493136,2025,2025-02-28,8042,Serveis,De 06:00:00 a 11:59:59 h,12113
1493137,2025,2025-02-28,8042,Serveis,De 12:00:00 a 17:59:59 h,12507
1493138,2025,2025-02-28,8042,Serveis,De 18:00:00 a 23:59:59 h,10013


In [103]:
df = df[df['Tram_Horari'] != 'No consta'].copy()
df['Hora_Inicio'] = df['Tram_Horari'].str.extract(r'De (\d{2}:\d{2}:\d{2})')
df['Fecha_Hora'] = pd.to_datetime(df['Data'] + ' ' + df['Hora_Inicio'])
df = df.drop(columns=['Tram_Horari', 'Hora_Inicio'])
df

,Any,Data,Codi_Postal,Sector_Economic,Valor,Fecha_Hora
0,2019,2019-01-01,8001,Indústria,727,2019-01-01 00:00:00
1,2019,2019-01-01,8001,Indústria,855,2019-01-01 06:00:00
2,2019,2019-01-01,8001,Indústria,1175,2019-01-01 12:00:00
3,2019,2019-01-01,8001,Indústria,1214,2019-01-01 18:00:00
5,2019,2019-01-01,8001,Residencial,28259,2019-01-01 00:00:00
...,...,...,...,...,...,...
1493133,2025,2025-02-28,8042,Residencial,31585,2025-02-28 18:00:00
1493135,2025,2025-02-28,8042,Serveis,6132,2025-02-28 00:00:00
1493136,2025,2025-02-28,8042,Serveis,12113,2025-02-28 06:00:00
1493137,2025,2025-02-28,8042,Serveis,12507,2025-02-28 12:00:00


In [107]:
weather = weather.rename(columns={'valid_time': 'Fecha_Hora'})
df_combined = df.merge(weather, on="Fecha_Hora", how="left")
df_combined['t2m'] = df_combined['t2m'] - 273.15
df_combined = df_combined.drop(columns=["Data", "Hora_Inicio"], errors="ignore")
df_combined

,Any,Codi_Postal,Sector_Economic,Valor,Fecha_Hora,t2m,tp
0,2019,8001,Indústria,727,2019-01-01 00:00:00,9.646149,0.000000
1,2019,8001,Indústria,855,2019-01-01 06:00:00,9.818512,0.000000
2,2019,8001,Indústria,1175,2019-01-01 12:00:00,14.156647,0.000000
3,2019,8001,Indústria,1214,2019-01-01 18:00:00,12.701080,0.000000
4,2019,8001,Residencial,28259,2019-01-01 00:00:00,9.646149,0.000000
...,...,...,...,...,...,...,...
1194507,2025,8042,Residencial,31585,2025-02-28 18:00:00,11.339508,0.000000
1194508,2025,8042,Serveis,6132,2025-02-28 00:00:00,11.016998,0.000000
1194509,2025,8042,Serveis,12113,2025-02-28 06:00:00,9.402496,0.000007
1194510,2025,8042,Serveis,12507,2025-02-28 12:00:00,13.221832,0.000004


In [108]:
df_combined.isnull().sum()

Any                0
Codi_Postal        0
Sector_Economic    0
Valor              0
Fecha_Hora         0
t2m                0
tp                 0
dtype: int64

In [110]:
df_combined.to_csv('barcelona.csv', index=False)